In [1]:
import sys
# sys.path.append("/Users/bubble/Desktop/Project/Infrasound Sensor/Layout/Code/acousticsensor")

import gdsfactory as gf
# TODO
# 1. Change one side of the cell_temp to be comparation with the width
# 2. Different with the width and same width with gap

cell_temp = gf.Component()

2025-03-13 14:01:31.932 | INFO     | kfactory.kcell:show:8777 - klive v0.3.3: Opened file '/Users/bubble/Desktop/Project/Infrasound Sensor/Layout/Code/acousticsensor/build/gds/2815967668.oas'


In [2]:
# cell
L = 3000
w = [1, 2, 4, 8, 16, 32, 64, 128]

block = gf.Component()
origin = (5000 / len(w), 0)

for i in range(len(w)):
    beam = gf.components.rectangle(size=(w[i], L), layer=(9, 0))
    beam_ref = block << beam
    beam_ref.movex(5000/(len(w)+1)*(i+1)-w[i]/2)
    # length mark
    T = gf.components.text(f"{w[i]}", size=50, layer=(1, 0))
    T_ref = block << T
    T_ref.move(((5000/(len(w)+1)*(i+1)-w[i]/2), -150))
            



# block.show()

In [3]:
# backside etching 5mm * 5mm (5743.44um)
backside = gf.Component()
size = 5743.44
diff = (size - 5000) / 2
for i in range(2):
    backside_temp = gf.components.rectangle(size=(size, size), layer=(3, 0))
    backside_ref = backside << backside_temp
    if i == 0:
        backside_ref.move((-diff, -diff))
    elif i == 1:
        backside_ref.move((10000-diff, -diff))

# backside.show()

In [4]:
# structure for each die
# structure for a block
fblock = gf.Component()
for i in range(2):
    block_ref = fblock << block
    if i == 0:
        pass
    elif i == 1:
        block_ref = fblock << block
        block_ref.move((10000, 0))


In [5]:
# order
order = gf.Component()
text_array = []
for i in range(2):
    text_array.append(gf.Component())

for i in range(2):
    T = gf.components.text(f"BeL2 {i+1}", size=20, layer=(1, 0))
    for j in range(4):
        order_ref = text_array[i] << T
        if j == 0:
            order_ref.move((-100, 0))
        elif j == 1:
            order_ref.move((-100, 5000))
        elif j == 2:
            order_ref.move((5020, 0))
        else:
            order_ref.move((5020, 5000))
    text_array_ref = order << text_array[i]
    if i == 0:
        text_array_ref.move((0, 0))
    elif i == 1:
        text_array_ref.move((10000, 0))





In [6]:
# backside frame 5mm * 5mm 
backframe = gf.Component()
size = 5000
for i in range(2):
    backframe_temp = gf.components.rectangle(size=(size, size), layer=(3, 0))
    backframe_ref = backframe << backframe_temp
    if i == 0:
        pass
    elif i == 1:
        backframe_ref.move((10000, 0))

# backside.show()

In [7]:
# boolean operation
# do backside etching - frame
outside = gf.boolean(A = backframe, B = fblock,  operation="not", layer1=(3, 0), layer2=(9, 0), layer=(2, 0))
outside = gf.boolean(A = outside, B = fblock,  operation="not", layer1=(2, 0), layer2=(10, 0), layer=(1, 0))

cell_temp << outside

# add holes to cell_temp
holes = gf.boolean(A = fblock, B = fblock,  operation="not", layer1=(8, 0), layer2=(10, 0), layer=(1, 0))
cell_temp << holes
# add holes to cell_temp
pc = gf.boolean(A = fblock, B = fblock,  operation="not", layer1=(11, 0), layer2=(30, 0), layer=(2, 0))
cell_temp << pc
cell_temp << backside
# add order
marker = gf.boolean(A = fblock, B = fblock,  operation="not", layer1=(1, 0), layer2=(10, 0), layer=(1, 0))
cell_temp << marker
cell_temp << order
# frame
frame1 = gf.components.rectangle(size=(15000, 5000), layer=(20, 0))
cell_temp << frame1
frame2 = gf.components.rectangle(size=(20000, 15830), layer=(21, 0))
frame2_ref = cell_temp << frame2
frame2_ref.move((-2500, -5415))

cell_beam_long3 = gf.Component()
cell_ref = cell_beam_long3 << cell_temp
cell_ref.move((2500, 5415-10000))
cell_beam_long3.show()
# cell_beam_long3.write_gds("mesh.gds")
# cell_beam_long3.plot()